# Launch TSWAP algorithm

This Jupyther notebook runs the TSWAP algorithm on a specified XML tasks. 

This code iterates over a set of predefined maps to run a series of experiments with the TSWAP algorithm. For each map, it generates results for a range of agent numbers, logs experiment details, and stores the results in a text file. 

On each step it reads information on grid maps, start and goal configurations for agents and executing the TSWAP solver with these parameters. As a result, for each run next indicators will be saved to the XML file:

1.	`number` - The number of agents involved in the experiment.
2.	`collision` - The total number of agent collisions, which is 0 for TSWAP algorithm (because this algorithm guaratees collision free movement).
3.	`collision_obst` - The total number of collisions between agents and obstacles, also 0.
4.	`flowtime` - The total flowtime of the solution, representing the cumulative time all agents take to reach their goals.
5.	`makespan` - The makespan of the solution, representing the time taken until the last agent reaches its goal.
6.	`runtime` - The execution time of the TSWAP algorithm for this experiment, recorded in seconds.
7.	`success` - The success rate of the experiment, where 100 represents success (solved * 100).
8.	`sum_of_dists` - The total distance traveled by all agents, calculated as the sum of individual agent travel distances.
9.	`max_dist` - The maximum distance traveled by any single agent.

**Notes**

This notebook requires the TSWAP executable and associated directories to be properly set up using variables `TSWAP_DIR`, `TSWAP_BUILD_DIR`, etc.

In [1]:
import numpy as np
import os
from datetime import datetime
import manavlib.io.xml_io as xml_io
import manavlib.io.tswap_io as tswap_io
import manavlib.io.movingai_io as movingai_io
from pathlib import Path
from typing import Tuple

In [2]:
TSWAP_DIR = "../tswap"  # Path to the cloned TSWAP repository (https://github.com/Kei18/tswap). Modify this path to point to your local TSWAP repository.

TSWAP_BUILD_DIR = "build"  # Directory within TSWAP_DIR where the TSWAP project has been built. Modify this if your build location differs.

XML_TASKS_DIR = "tasks"  # Directory containing XML files with task and map configurations. Modify this to point to your XML tasks directory.

MAPS_NAMES = [
    "random-32-32-10",
    "room-32-32-4",
]  # Names of subdirectories in XML_TASKS_DIR where tasks and map files are stored. Additional map names can be added to this list.


EXPERIMENT_RESULTS_DIR = "results"  # Directory for storing experiment result files. 

# TSWAP settings
EXPERIMENTS_MAX_STEPS = 1000  # Maximum number of steps allowed per experiment run.
EXPERIMENTS_MAX_TIME = 1000  # Maximum runtime in seconds for each experiment.
TSWAP_METHOD = "TSWAP"  # The name of the planning method to use. Refer to the TSWAP repository for more information.


# Fixed settings (do not modify when following the standard instructions)
TSWAP_EXE = "app"  # Name of the TSWAP executable binary file after the build.
TSWAP_MAP_DIR = "map" # Subdirectory within TSWAP_DIR where map files are stored.
XML_TASK_SUFFIX = "_task"  # Suffix of task files in the XML format. 
TSWAP_OUTPUT_FILE_PREFIX = "tswap_result_"  # Prefix for naming output result files from the TSWAP algorithm.

In [3]:
agents_from = 10  # Starting number of agents for the experiment.
agents_to = 50    # Maximum number of agents for the experiment.
agents_step = 10  # Increment step for the number of agents in each experiment run.
task_num = 10     # Number of tasks to run for each map.

In [4]:
def run_tswap_on_xml_task(
    xml_task_path: str,
    xml_task_name: str,
    map_name: str,
    agents_num: int,
    max_steps: int,
    max_time: float,
    hide_output: bool,
    save_xml_log: bool,
) -> Tuple[bool, int, int, float, int, int, float, float]:
    """
    Runs the TSWAP algorithm on a specified XML task.

    Every launch of TSWAP algorithm contains the next steps:

    1. **XML Input Processing**: Reads agent parameters (e.g., size, positions, goals) and map
        information from XML files.
    2. **TSWAP Setup**: Creates an instance file with map and agent configurations for the TSWAP algorithm.
    3. **TSWAP Execution**: Runs the TSWAP solver and retrieves results from the output log file.
    4. **Solution Processing**: Converts the agent solution positions from grid to real-world coordinates and calculates the main indicators (makespan, flowtime, sum of traveled distances and maximum distance).

    Parameters
    ----------
    xml_task_path : str
        Path to the directory containing the XML task files.
    xml_task_name : str
        Name of the XML task file (without extension) that defines agent parameters.
    map_name : str
        Name of the map file containing grid layout information.
    agents_num : int
        Number of agents involved in the task.
    max_steps : int
        Maximum number of steps allowed for the TSWAP algorithm to execute.
    max_time : float
        Maximum allowed computation time for the TSWAP algorithm.
    hide_output : bool
        If True, suppresses console output from the TSWAP executable.
    save_xml_log : bool
        If True, saves the solution as an XML log file (not implemented yet).

    Returns
    -------
    Tuple[bool, int, int, float, int, int, float, float]
        A tuple containing:
        - `solved` (bool): Indicates if a solution was found.
        - `flowtime` (int): Flowtime (duration in steps) of the solution.
        - `makespan` (int): Makespan (sum of durations in steps) of the solution.
        - `runtime` (float): Execution time of the TSWAP algorithm (in seconds).
        - `sum_of_dists` (float): Sum of distances traveled by all agents.
        - `max_dist` (float): Maximum distance traveled by a single agent.
    """

    default_params, agents_positions, agents_goals, agents_params = (
        xml_io.read_xml_agents(os.path.join(xml_task_path, f"{xml_task_name}.xml"))
    )
    width, height, cell_size, occupancy_grid = xml_io.read_xml_map(
        os.path.join(xml_task_path, map_name)
    )
    default_size = default_params.size
    time_step = 1
    default_speed = 1

    if default_size / cell_size > (2**0.5) / 4:
        print("Agents size too large for simultaneous movement on grid")
        return False

    map_file = f"{xml_task_name}_{agents_num}.map"
    map_path = os.path.join(TSWAP_DIR, TSWAP_MAP_DIR, map_file)
    instance_file = f"{xml_task_name}_{agents_num}.txt"
    instance_path = os.path.join(TSWAP_DIR, instance_file)
    movingai_io.create_map_file(map_path, height, width, occupancy_grid)
    tswap_starts = tswap_io.convert_ij_positions_to_tswap(agents_positions)
    tswap_goals = tswap_io.convert_ij_positions_to_tswap(agents_goals)

    tswap_io.create_tswap_instance_file(
        instance_path,
        map_file,
        agents_num,
        max_steps,
        max_time,
        tswap_starts,
        tswap_goals,
    )
    tswap_exe_path = os.path.join(TSWAP_DIR, TSWAP_BUILD_DIR, TSWAP_EXE)
    tswap_output_file = f"{xml_task_name}_{agents_num}_result.txt"
    tswap_output_path = os.path.join(TSWAP_DIR, tswap_output_file)
    tswap_output = "> /dev/null" if hide_output else "-v > output.txt"

    tswap_command = f"{tswap_exe_path} -i {instance_path} -s {TSWAP_METHOD} -o {tswap_output_path} {tswap_output}"
    os.system(tswap_command)
    agents_num, solved, flowtime, makespan, runtime, solution = tswap_io.read_tswap_log(
        tswap_output_path
    )
    timesteps_in_step = int(cell_size / (default_speed * time_step))
    flowtime_timesteps = flowtime * timesteps_in_step
    makespan_timesteps = makespan * timesteps_in_step
    transition_distances = np.zeros(agents_num, dtype=np.float64)
    solution_real_positions = np.zeros((agents_num, makespan_timesteps, 2))
    for agent_id, agent_solution in enumerate(solution):
        agent_solution_real = tswap_io.convert_tswap_to_real_positions(
            agent_solution, height, cell_size
        )
        for step in range(makespan):
            pos_from = agent_solution_real[step]
            pos_to = agent_solution_real[step + 1]
            solution_real_positions[
                agent_id, step * timesteps_in_step : (step + 1) * timesteps_in_step, 0
            ] = np.linspace(
                pos_from[0],
                pos_to[0],
                timesteps_in_step,
                endpoint=False,
                dtype=np.float64,
            )
            solution_real_positions[
                agent_id, step * timesteps_in_step : (step + 1) * timesteps_in_step, 1
            ] = np.linspace(
                pos_from[1],
                pos_to[1],
                timesteps_in_step,
                endpoint=False,
                dtype=np.float64,
            )
            transition_distances[agent_id] += (
                tuple(agent_solution[step]) != tuple(agent_solution[step + 1])
            ) * cell_size

    sum_of_dists = np.sum(transition_distances)
    max_dist = np.max(transition_distances)
    # TODO
    # if save_xml_log:
    #     tswap_log_file = f"{xml_task_name}_{agents_num}_tswap_log.xml"
    #     tswap_log_path = os.path.join(os.path.dirname(xml_task_path), tswap_log_file)
    #     print("Log saved", tswap_log_path)
    #     xml_io.create_log_file(xml_task_path, tswap_log_path, solution_real_positions)
    print(
        solved,
        flowtime,
        makespan,
        runtime,
        flowtime_timesteps,
        makespan_timesteps,
        sum_of_dists,
        max_dist,
    )
    return (
        solved,
        flowtime,
        makespan,
        runtime,
        flowtime_timesteps,
        makespan_timesteps,
        sum_of_dists,
        max_dist,
    )

In [ ]:
run_tswap_on_xml_task(
    os.path.join(XML_TASKS_DIR, MAPS_NAMES[0]),
    "0_task",
    "map.xml",
    110,
    5000,
    5000,
    True,
    False,
)

In [ ]:
now = datetime.now()
now_str = now.strftime("%d_%m_%y_%H_%M_%S")

for map_name in MAPS_NAMES:

    series_result_file = f"result_{now_str}.txt"
    series_map_result_dir = os.path.join(EXPERIMENT_RESULTS_DIR, map_name, "orig-tswap")
    print(series_map_result_dir)
    if not os.path.exists(series_map_result_dir):
        path = Path(series_map_result_dir)
        path.mkdir(parents=True)

    series_result_path = os.path.join(series_map_result_dir, series_result_file)
    results_log = open(series_result_path, "w")
    header_str = (
        "{:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20}\n".format(
            "number",
            "collision",
            "collision_obst",
            "flowtime",
            "makespan",
            "runtime",
            "success",
            "sum_of_dists",
            "max_dist",
        )
    )
    results_log.write(header_str)
    for agents_num in range(agents_from, agents_to + 1, agents_step):
        for task_id in range(task_num):
            xml_task_path = os.path.join(XML_TASKS_DIR, map_name)
            (
                solved,
                flowtime,
                makespan,
                runtime,
                flowtime_timesteps,
                makespan_timesteps,
                sum_of_dists,
                max_dist,
            ) = run_tswap_on_xml_task(
                xml_task_path,
                f"{task_id}{XML_TASK_SUFFIX}",
                "map.xml",
                agents_num,
                EXPERIMENTS_MAX_STEPS,
                EXPERIMENTS_MAX_TIME,
                True,
                False,
            )
            curr_res_str = "{:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20}\n".format(
                agents_num,
                0,
                0,
                flowtime,
                makespan,
                runtime / 1000,
                solved * 100,
                sum_of_dists,
                max_dist,
            )
            results_log.write(curr_res_str)
    results_log.close()